In [ ]:
import json 
import boto3

In [ ]:
config_data = json.load(open('config.json'))
aws_access_key_id = config_data['aws_access_key_id']
aws_secret_access_key = config_data['aws_secret_access_key']
config_data.keys()

In [ ]:
# create a s3 client
s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

In [ ]:
## list all folder in a s3 bucket 
import boto3

def list_s3_folders(bucket_name):
    s3_client = boto3.client('s3')
    
    # List objects in the bucket
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name)
    
    folders = set()
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                key = obj['Key']
                # Extract the folder name
                folder = key.split('/')[0]
                folders.add(folder)
    
    return sorted(folders)

bucket_name = 'bindcraft'
folders = list_s3_folders(bucket_name)
print("Folders in S3 bucket:")
for folder in folders:
    print(folder)

In [ ]:
import boto3
import os

def download_s3_folder(bucket_name, s3_folder, local_dir):
    s3_client = boto3.client('s3')
    
    # Ensure the local directory exists
    if not os.path.exists(local_dir):
        os.makedirs(local_dir)
    
    # List objects in the specified S3 folder
    paginator = s3_client.get_paginator('list_objects_v2')
    pages = paginator.paginate(Bucket=bucket_name, Prefix=s3_folder)
    
    for page in pages:
        if 'Contents' in page:
            for obj in page['Contents']:
                s3_key = obj['Key']
                local_file_path = os.path.join(local_dir, os.path.relpath(s3_key, s3_folder))
                
                # Ensure the local directory exists
                local_file_dir = os.path.dirname(local_file_path)
                if not os.path.exists(local_file_dir):
                    os.makedirs(local_file_dir)
                
                # Download the file
                s3_client.download_file(bucket_name, s3_key, local_file_path)
                print(f"Downloaded {s3_key} to {local_file_path}")

# Example usage
bucket_name = 'bindcraft'
s3_folder = ''
local_dir = f'./out/bindcraft/{s3_folder}'

download_s3_folder(bucket_name, s3_folder, local_dir)